In [1]:
import sys
sys.path.insert(1, '../')

from river import stream,tree,metrics
from encoding import prefix_bin
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from collections import Counter

from tqdm import tqdm
import pickle as pkl
import matplotlib.pyplot as plt
import os,json
import datetime
from collections import deque, Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

import warnings

warnings.filterwarnings("ignore")
import xgboost as xgb

import datetime, time
import utils
import sliding_window
import psutil

import copy

In [2]:
def display_progress(row_counting, total_length, interval=2000):
    if rowcounter%interval == 0:
        print(round(rowcounter*100/totallength,2) ,'%', 'Case finished: %s'%(casecount), 'Running cases: %s'%(len(streaming_db)))

In [3]:
def update_test_cases(new_case, test_cases):
    test_cases.append(new_case)
    if len(test_cases) > test_size:
        test_cases.popleft()

In [12]:
# Experiment settings
'./DATA/logs/synthetic_log_bc1.csv'
dataset_label = 'IRO5K'

# Invoke parameters for dataset
window_size = 100
test_size = 30
gp = 200
training_rebalance = False
with open('../dataset_parameters.json','r') as json_file:
    parameters = json.load(json_file)[dataset_label]
    print(parameters)
    key_pair = parameters['key_pair']
    catatars = parameters['categorical_attrs']
    maximum_prefixs = parameters['maximum_prefix']


for maximum_prefix in range(1, maximum_prefixs+1):
    print(maximum_prefix)
    train_window_dict = {}
    test_window_dict = {}
    dataset_loc = '../DATA/logs/'+ dataset_label +'.csv'
    try:
        os.makedirs('../result/%s'%(dataset_label))
    except:
        pass

    # Set streaming classifier
    classifier = 'rf'
    if classifier == 'htc':
        streaming_classifier = tree.HoeffdingTreeClassifier(grace_period=10, split_criterion='info_gain')
    elif classifier =='hatc':
        streaming_classifier = tree.HoeffdingAdaptiveTreeClassifier(grace_period=10, split_criterion='info_gain' )
    elif classifier == 'efdt':
        streaming_classifier = tree.ExtremelyFastDecisionTreeClassifier(grace_period=10, split_criterion='info_gain')
    elif classifier == 'rf':
        streaming_classifier = RandomForestClassifier(n_estimators = 10, n_jobs = -1)
    elif classifier == 'xgb':
        streaming_classifier = xgb.XGBClassifier(n_estimators = 10, learning_rate=0.01, verbosity =0)     
    dataset = stream.iter_csv(
            dataset_loc
            )

    totallength = len(list(dataset))

    dataset = stream.iter_csv(
            dataset_loc,
            # drop=['Complete Timestamp'],
            target='outcome'
            )
    enctype = 'Index-base'
    
    streaming_db ={}
    training_windows = sliding_window.training_window(window_size,test_size)
    training_models ={}
    test_cases = deque()
    feature_matrix ={}

    save_model = {}
    casecount = 0
    rowcounter = 0
    finished_db ={}
    running_case = 0
    window_acc_dict = {}
    prediction_result = dict()
    for i in range(1, maximum_prefix+1): prediction_result[i] = {}
    finished_caseid = set()
    usedingrace = set()
    
    
    '''
    Before test and training streaming event predictive monitoring, grace period is preceded to initialize model by prefix length
    and obtain feature matrix to transform future events
    '''
    for x,y in dataset:
#         display_progress(rowcounter, totallength)
        rowcounter +=1
        # Event stream change dictionary keys
        x = utils.dictkey_chg(x, key_pair)

        if dataset_label !='bpic15':
            x['ts'] = x['ts'][:-4]

        x['outcome'] =y 
        # Initialize case by prefix length        
        caseid = x['caseid']
        outcome = x['outcome']
    #     progress = x['progress']

        x.pop('caseid')
        x.pop('outcome')

    #     x.pop('progress')

        case_bin = prefix_bin(caseid, x)
        case_bin.set_enctype(enctype)

        if caseid not in list(streaming_db.keys()):
            case_bin.set_prefix_length(1)    
            streaming_db[caseid] = []
        elif caseid in finished_caseid:
            pass
        else:
            case_bin.set_prefix_length(len(streaming_db[caseid])+1)
            case_bin.set_prev_enc(streaming_db[caseid][-1])

        # Encode event and cases and add to DB
        case_bin.update_truelabel(outcome)   
        case_bin.update_encoded(catattrs=catatars,enctype=enctype)
        ts = case_bin.event['ts']
        streaming_db[caseid].append(case_bin)
        # Detect label appeared case 
        if outcome != '' and caseid not in finished_caseid:
            usedingrace.add(caseid)
            for i in streaming_db[caseid]:
                i.update_truelabel(outcome)
            finished_caseid.add(caseid)
            # Adding newly finished case to training set.    
            casecount +=1
            # Grace period to collect feature matrix
            if casecount < gp:
                case_length = len(streaming_db[caseid])
                if case_length >= maximum_prefix:
                    if 'prefix_%s'%(maximum_prefix) not in list(feature_matrix.keys()):
                        feature_matrix['prefix_%s'%(maximum_prefix)]=set()
                        training_models['prefix_%s'%(maximum_prefix)] = [streaming_classifier,
                                                                   0]
                    feature_list = list(streaming_db[caseid][maximum_prefix-1].encoded.keys())
                    for x in feature_list: feature_matrix['prefix_%s'%(maximum_prefix)].add(x) 
            else:
                break
                
    if casecount not in train_window_dict.keys(): train_window_dict[casecount] = []
    if casecount not in test_window_dict.keys(): test_window_dict[casecount] = []

    for caseid in list(usedingrace):
        case_length = len(streaming_db[caseid])
        if case_length >= maximum_prefix:
            x = streaming_db[caseid][maximum_prefix-1]
            if x.prefix_length != 0:            
                training_windows.update_window(x)
    
            update_test_cases(streaming_db[caseid], test_cases)
    train_window_dict[casecount].append(copy.deepcopy(training_windows.container))
    
    x_prefix_length = 0
    training_x = []
    training_y = []
    for pos, i in enumerate(training_windows.container):
        x_prefix_length = i.prefix_length 
        i.encoded = utils.readjustment_training(i.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
        training_x.append(i.encoded)
        training_y.append(i.true_label)

    
    le = LabelEncoder()
    training_y = le.fit_transform(training_y)
    training_x = pd.DataFrame.from_dict(training_x)
    
    ###
    #Oversampling
    ###
    if training_rebalance == True:
        smote = SMOTE()
        training_x, training_y = smote.fit_resample(training_x, training_y)
        
    training_models['prefix_%s'%(x_prefix_length)][0].fit(training_x, training_y)
    training_models['prefix_%s'%(x_prefix_length)][1] =casecount
    
    prediction_result[maximum_prefix][casecount] = {}
    y_truelist = []
    y_predlist = []
    for case in test_cases:
        if len(case) >= maximum_prefix:
            x = case[maximum_prefix-1]
            if x.prefix_length != 0:            
                model = training_models['prefix_%s'%(x_prefix_length)][0]
                current_event = utils.readjustment_training(x.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
                current_event = pd.Series(current_event).to_frame().T
                y_pred = training_models['prefix_%s'%(x_prefix_length)][0].predict_proba(current_event)
                y_truelist.append(x.true_label)
                y_predlist.append(y_pred)
                test_window_dict[casecount].append(x)
    prediction_result[maximum_prefix][casecount]['y_true'] = y_truelist
    prediction_result[maximum_prefix][casecount]['y_pred'] = y_predlist
    

    '''
    Streaming event label prediction start.
    - Test and training steps are executed when case finished/ event arrived with label
    '''
    for i in streaming_db.keys(): usedingrace.add(i)
    streaming_db ={}
    cdhappend ={}
    for i in range(1, maximum_prefix+1): cdhappend[i] = 0
    for x,y in dataset:
        display_progress(rowcounter, totallength)

        rowcounter +=1
        # Event stream change dictionary keys
        x = utils.dictkey_chg(x, key_pair)

        if dataset_label !='bpic15':
            x['ts'] = x['ts'][:-4]

        # Check label possible
        # x = utils.set_label(x)
        x['outcome'] =y 
        # Initialize case by prefix length
        caseid = x['caseid']
        outcome = x['outcome']
        x.pop('caseid')
        x.pop('outcome')

        if caseid not in usedingrace:
            case_bin = prefix_bin(caseid, x)
            case_bin.set_enctype(enctype)

            if caseid not in list(streaming_db.keys()):
                case_bin.set_prefix_length(1)    
                streaming_db[caseid] = []
                running_case +=1
            elif caseid in finished_caseid:
                pass
            else:
                case_bin.set_prefix_length(len(streaming_db[caseid])+1)
                case_bin.set_prev_enc(streaming_db[caseid][-1])

            # Encode event and cases and add to DB
            case_bin.update_truelabel(outcome)   
            case_bin.update_encoded(catattrs=catatars,enctype=enctype)
            ts = case_bin.event['ts']

            if case_bin.prefix_length == maximum_prefix:
                case_bin.encoded = utils.readjustment_training(case_bin.encoded, feature_matrix['prefix_%s'%(case_bin.prefix_length)])
            streaming_db[caseid].append(case_bin)

            # Detect label appeared case 
            if outcome != '' and caseid not in finished_caseid:
                finished_caseid.add(caseid)

                # Adding newly finished case to training set.
                casecount +=1    

                # Modify encoded attributes of cases with feature matrix
                case_length = len(streaming_db[caseid])
                if case_length >= maximum_prefix:

                    update_test_cases(streaming_db[caseid], test_cases)
                    streaming_db[caseid][maximum_prefix-1].update_truelabel(outcome)
                    x = streaming_db[caseid][maximum_prefix-1].encoded
                    prefix_length =streaming_db[caseid][maximum_prefix-1].prefix_length                    
#                     save_model['%s_%s'%(prefix_length, model_update_count)] = training_models['prefix_%s'%(streaming_db[caseid][maximum_prefix-1].prefix_length)][0]

                y_truelist = []
                y_predlist = []

                if casecount not in test_window_dict.keys():
                    test_window_dict[casecount] = []
                    
                for case in test_cases:
                    if len(case) >= maximum_prefix:
                        x = case[maximum_prefix-1]
                        if x.prefix_length != 0:
                            length = x.prefix_length
                            current_event = utils.readjustment_training(x.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
                            current_event = pd.Series(current_event).to_frame().T
                            
                            if casecount != gp:
                                c_id = [x.caseid for x in test_window_dict[casecount-1]]
                                
                            if x.caseid not in c_id:
                                y_pred = training_models['prefix_%s'%(x_prefix_length)][0].predict_proba(current_event)
                            else:
                                y_pred = prediction_result[maximum_prefix][casecount-1]['y_pred'][c_id.index(x.caseid)]
                            y_truelist.append(x.true_label)
                            y_predlist.append(y_pred)
                            
                            if casecount not in test_window_dict.keys():
                                test_window_dict[casecount] = []
                            test_window_dict[casecount].append(x)
                prediction_result[maximum_prefix][casecount] = {}
                prediction_result[maximum_prefix][casecount]['y_true'] = y_truelist
                prediction_result[maximum_prefix][casecount]['y_pred'] = y_predlist
#                 if 'b1' not in caseid and cdhappend[maximum_prefix] == 0:
#                     cdhappend[maximum_prefix] = model_update_count
    
    try:
        os.makedirs('../result/%s/%s/Finished cases/'%(dataset_label, classifier))
    except:
        pass    
    
    with open('../result/%s/%s/Finished cases/train_rebalance_%s prefix_%s training window.pkl'%(dataset_label, classifier, str(training_rebalance), maximum_prefix), 'wb') as f:
        pkl.dump(train_window_dict, f)
    with open('../result/%s/%s/Finished cases/train_rebalance_%s prefix_%s test window.pkl'%(dataset_label, classifier, str(training_rebalance), maximum_prefix), 'wb') as f:
        pkl.dump(test_window_dict, f)

    with open('../result/%s/%s/Finished cases/train_rebalance_%s prefix_%s update.pkl'%(dataset_label, classifier, str(training_rebalance), maximum_prefix), 'wb') as f:
        pkl.dump(prediction_result, f)



{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Complete Timestamp': 'ts'}, 'categorical_attrs': ['activity'], 'maximum_prefix': 12}
1
37.19 % Case finished: 375 Running cases: 181
55.78 % Case finished: 562 Running cases: 368
74.38 % Case finished: 739 Running cases: 548
92.97 % Case finished: 917 Running cases: 727
2
37.19 % Case finished: 375 Running cases: 181
55.78 % Case finished: 562 Running cases: 368
74.38 % Case finished: 739 Running cases: 548
92.97 % Case finished: 917 Running cases: 727
3
37.19 % Case finished: 375 Running cases: 181
55.78 % Case finished: 562 Running cases: 368
74.38 % Case finished: 739 Running cases: 548
92.97 % Case finished: 917 Running cases: 727
4
37.19 % Case finished: 375 Running cases: 181
55.78 % Case finished: 562 Running cases: 368
74.38 % Case finished: 739 Running cases: 548
92.97 % Case finished: 917 Running cases: 727
5
37.19 % Case finished: 375 Running cases: 181
55.78 % Case finished: 562 Running cases: 368
74.38 % Case fini

In [39]:
'../result/%s/%s/Finished cases/train_rebalance_%s prefix_%s update.pkl'%(dataset_label, classifier, str(training_rebalance), maximum_prefix)

'../result/IRO5K/xgb/Finished cases/train_rebalance_True prefix_12 update.pkl'